# Ting_Zhang_Homework 2

## Reading
* Descriptor: https://realpython.com/python-descriptors/#how-attributes-are-accessed-with-the-lookup-chain
* Data model: https://docs.python.org/3/reference/datamodel.html (contents related to what we taught)
* Python MRO: https://www.python.org/download/releases/2.3/mro/

## Merge sorted stream
Write a generator function that takes two sorted streams (generators), and return a generator that can produce a merged stream in sorted order.

Bonus point: can you make it generic such that it can merge any number of streams?


```python
def merge_sorted_stream(stream1, stream2):
    yield ...

stream1 = range(0, 10, 2)
stream2 = range(1, 10, 2)

for x in merge_sorted_stream(stream1, stream2):
    print(x)

0 
1
2
3
4
5
6
7
8
9
```

In [25]:
def merge_sorted_stream(*args):
    iters = [iter(arg) for arg in args]
    
    nums = [None] * len(iters)
    #print(nums)
    for x, it in enumerate(iters):
        try:
            nums[x] = next(it)
        except StopIteration:
            nums[x] = None
            
    while True:
        new_nums = [num for num in nums if num is not None]
        #print(new_nums)
        if len(new_nums) != 0:
            min_num = min(new_nums)
            min_index = nums.index(min_num)
            try:
                yield min_num
                nums[min_index] = next(iters[min_index])
            except StopIteration:
                nums[min_index] = None  
        else:
            break           

In [28]:
stream1 = range(0, 10, 2)
stream2 = range(1, 10, 2)

for x in merge_sorted_stream(stream1, stream2):
    print(x)


0
1
2
3
4
5
6
7
8
9


In [29]:
stream3 = range(1, 10, 3)
for x in merge_sorted_stream(stream1, stream2, stream3):
    print(x)

0
1
1
2
3
4
4
5
6
7
7
8
9


## Tree traversal

Unlike linear data structures (Array, Linked List, Queues, Stacks, etc) which have only one logical way to traverse them, trees can be traversed in different ways. Following are the generally used ways for traversing trees.

```
      1
    /  \ 
   2    3
  / \
 4   5
```

Depth First Traversals: 
  * (a) Inorder (Left, Root, Right) : 4 2 5 1 3
  * (b) Preorder (Root, Left, Right) : 1 2 4 5 3
  * (c) Postorder (Left, Right, Root) : 4 5 2 3 1

Define a Tree class with a method that can walk through the tree in different orders. Hint: use generator will make your life a lot easier.

```python
class TreeNode:
    
    ...

    def in_order(self):
        pass
        
    def pre_order(self):
        pass
        
    def post_order(self):
        pass

    
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

>>> print(' -> '.join(item for item in root.in_order()))
4 -> 2 -> 5 -> 1 -> 3    


```
   

In [32]:
class TreeNode:
    
    def __init__(self, x):
        self.info = str(x)
        self.left = None
        self.right = None

    def in_order(self):
        #Left
        if self.left != None:
            for i in self.left.in_order():
                yield i
        #Root        
        yield self.info
        #Right
        if self.right != None:
            for i in self.right.in_order():
                yield i
        
    def pre_order(self):
        #Root        
        yield self.info
        #Left
        if self.left != None:
            for i in self.left.pre_order():
                yield i
        #Right
        if self.right != None:
            for i in self.right.pre_order():
                yield i
        
    def post_order(self):
        #Left
        if self.left != None:
            for i in self.left.post_order():
                yield i
        #Right
        if self.right != None:
            for i in self.right.post_order():
                yield i
        #Root        
        yield self.info

In [34]:
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

print('in order')
print(' -> '.join(item for item in root.in_order()))

print('pre order')
print(' -> '.join(item for item in root.pre_order()))

print('post order')
print(' -> '.join(item for item in root.post_order()))

in order
4 -> 2 -> 5 -> 1 -> 3
pre order
1 -> 2 -> 4 -> 5 -> 3
post order
4 -> 5 -> 2 -> 3 -> 1


## Implement a timer
Implement a timer that can print the execution time of your code. Try to implement it both as a decorator and as a context manager to compare the implementations. Can you implement it using one single class? 

Example:
```python
import time


with timer() as timer:
    time.sleep(3)

# Total execution time: 3.000123 seconds (a made up number)


@timer
def sleep(secs):
    time.sleep(secs)
    
sleep(3)
# Total execution time: 3.000123 seconds (a made up number)
```

Below is the code snippet to measure the time:
```python    
import time
start_time = time.time()
main()
print(f"--- {time.time() - start_time} seconds ---"
```

In [80]:
import time
import functools

class timer:
    
    def __init__(self):
        self.start_time = time.time()
        
    def __enter__(self):
        self.start_time = time.time()
        return self
    
    def __call__(self, func):
        @functools.wraps(func)
        def inner(*args):
            self.start_time = time.time()
            res = func(*args)
            print(f"Total execution time: {time.time() - self.start_time} seconds")
            return res
    
        return inner
    
    def __exit__(self, *exc):
        print(f"Total execution time: {time.time() - self.start_time} seconds")
        return True

In [81]:
#as a context manager
with timer() as timer:
    time.sleep(3)

Total execution time: 3.0043070316314697 seconds


In [82]:
#as a decorator
@timer

def sleep(secs):
    time.sleep(secs)

sleep(3)

Total execution time: 3.005256175994873 seconds
